# Résolution Radiosité

Cette partie qui concerne la résolution du système d'équation est très directe. L'algorithme suivi est :

- trouver la surface i pour laquelle la radiosité qui "reste à émettre" (unshot) AiBi est la plus forte (c'est elle qui contribuera le plus à l'éclairage de la scène)
- pour toutes les autres surface j
    - mettre à jour ce qui est reçu en fonction de ce qui est émis par i et du facteur de forme
- et boucler jusqu'à ce qu'on atteigne un état d'équilibre

## Imports

Note: retour de pandas pour un affichage final

In [1]:
import numpy as np
import random

# Representation
import pandas as pd
from IPython.display import display, HTML
print("imports: OK")

rng = random.SystemRandom()
print("random generator: OK")

imports: OK
random generator: OK


In [2]:
DEFAULT_RHO = 0.55
print(f"default reflectance rho value = {DEFAULT_RHO}")

default reflectance rho value = 0.55


## Affichage

In [3]:
# Display Array, descending y axis
def dumpdata(msg, data):
    print(f"## {msg} ##\n {pd.DataFrame(data).sort_index(ascending=False, axis=0)}\n")

def color_max_red(value):
    v = int(value)
    hexcolor = '#%02x%02x%02x' % (v, v, v)
    return 'background-color: {}'.format(hexcolor)

def convertRatioToRGB(source):
    for y in range(HEIGHT):
        for x in range(WIDTH):
            if (source[y, x] == 0.):
                lighting_color[y, x] = 0
            else:
                lighting_color[y, x] = int(abs(source[y, x]) * 255)
                
def fakeDisplay(msg="", style=True):
    df = pd.DataFrame(lighting_color).sort_index(ascending=False, axis=0)
    
    if style:
        s = df.style.applymap(color_max_red)
    else:
        s = df.style
    if msg!="":
        display(HTML("<b>"+msg+"</b>"))
    display(HTML(s.render()+"</br>"))

## Surface

Les modifications :

- dans le constructeur
    - un changement cosmétique sur l'aquisition de l'identifiant
    - ρ : l'indice de diffusion de la surface
    - E : l'éclairement
    - B : la radiosité (exitance)
    - dB (ΔB) : la partie non encore émise de la radiosité associée à la surface
- un moyen de fixer E, B et dB après la création de l'objet

La réflectance rho est fixé à 0.8 par défaut

In [4]:
class Rect():
    count = -1

    def next():
        Rect.count += 1
        return Rect.count

    def __init__(self, cmin, cmax, norm, rho=DEFAULT_RHO, E=0.):
        self.idt = Rect.next()
        
        # Radiosity
        self.rho = rho
        self.E = E
        self.B = E
        self.dB = E

        # Geometry
        self.cmin = cmin
        self.cmax = cmax
        self.norm = norm
        self.lx = cmax[0] - cmin[0]
        self.ly = cmax[1] - cmin[1]
        self.lz = cmax[2] - cmin[2]
        self.area = self.lx*self.ly + self.lx*self.lz + self.ly*self.lz

        # Patchs
        self.split(0.5,0.5,0.5)
        print(f"{self.idt} {self.cmin} {self.cmax} area:{self.area}")

    def initLightSource(self, E, rho=0.):
        self.E = E
        self.B = E
        self.dB = E
        self.rho = rho
    
    def resetRadiosity(self, rho=DEFAULT_RHO):
        self.B = self.E
        self.dE = self.E
        self.rho = rho


    def getRndUV(self, umin, umax, vmin, vmax):
        # TODO: p have to be in ]0;1[ to avoid the r = 0 case (in Ri T Rj)
        pu = rng.uniform(umin, umax)
        pv = rng.uniform(vmin, vmax)
        #print(f"[get2Rnd] u({umin},{umax}) v({vmin}, {vmax}) {pu}, {pv}")
        return (pu, pv)

    def getRandomPoint(self):
        if (self.lz == 0):
            # R xOy
            (px, py) = self.getRndUV(self.cmin[0], self.cmax[0], self.cmin[1], self.cmax[1])
            pz = self.cmin[2]
        elif (self.ly == 0):
            # R xOz
            (px, pz) = self.getRndUV(self.cmin[0], self.cmax[0], self.cmin[2], self.cmax[2])
            py = self.cmin[1]
        else:
            # R yOz
            (py, pz) = self.getRndUV(self.cmin[1], self.cmax[1], self.cmin[2], self.cmax[2])
            px = self.cmin[0]
        #print(f"(idt:{self.idt} return point ({px}, {py}, {pz})");
        return(np.array([px, py, pz]))

    def getPatchs(self):
        return self.patch

    def split(self, dx=0.5, dy=0.5, dz=0.5):
        self.patch = []

        # Hack to ensure we do at least 1 iteration
        lx = dx  if self.lx == 0 else self.lx
        ly = dy  if self.ly == 0 else self.ly
        lz = dz  if self.lz == 0 else self.lz

        # used to cancel splits on u, v, w
        kx = 0. if lx < 1. else 1.
        ky = 0. if ly < 1. else 1.
        kz = 0. if lz < 1. else 1.

        x0 = self.cmin[0]
        y0 = self.cmin[1]
        z0 = self.cmin[2]

        z = dz / 2.
        while z < lz:
            y = dy / 2.
            while y < ly:
                x = dx / 2.
                while x < lx:
                    #print(f"{x0+x*kx}, {y0+y*ky}, {z0+z*kz}")
                    self.patch.append(np.array([x0+x*kx, y0+y*ky, z0+z*kz]))
                    x += dx
                y += dy
            z += dz

    def initFormFactors(self, n):
        self.fij = np.zeros([n])

    def dumpFij(self):
        n = len(self.fij)
        for i in range(n):
            print(f"[{self.idt} -> {i}] Fij = {self.fij[i]}")
            
print("class Rectangle: OK")

class Rectangle: OK


## Maillage

- un index *maxUnshotIndex* qui pointe sur la surface dont B x A est le plus grand cette itération
- un moyen de réinitialiser les calculs précédents *resetRadiosity*
- une méthode pour rechercher la surface ayant la plus forte radiosité à émettre *initRadiosity*
- une méthode pour résoudre de manière itérative le système en suivant l'algorithme présenté en début de page *solveRadiosity*. La convergence est estimée sur la quantité totale de "radiosité" émise lors de cette itération avec un garde fou sur le nombre maximal d'itérations

### Optimisation(s)

- le calcul des facteurs dans sa version initiale occupe 90% du temps nécessaire à l'intégralité des tâches d'éclairage
- En notant que $F_{ji} = \frac{Ai}{Aj}F_{ij}$, on élimine la moitié des calculs (cf. _getFormFactors_uni_sq2()_)
- je ne sais pas pourquoi mais le calcul de la norme d'un vecteur (np.linalg.norm()) s'avère relativement couteux, on obtient un gain en le remplaçant explicitement par ce qu'il représente (somme ri.norm[k] nrij[k])
- Sur une machine de teste, on passe pour une grille 10x10, on passe de 13.8s à 4.2s

In [5]:
class Mesh():
    def __init__(self):
        self.poly = []
        # Radiosity
        self.maxUnshotIndex = -1

    def add(self, item):
        self.poly.append(item)

    def get(self, index):
        return self.poly[index]

    def resetRadiosity(self, rho=DEFAULT_RHO):
        for p in self.poly:
            p.resetRadiosity()
    
    def resetLights(self):
        for p in self.poly:
            p.E = 0.
            p.B = p.E
            p.dB = p.E
    
    def initRadiosity(self):
        # find poly with max E
        maxUnshot = 0.
        for p in self.poly:
            unshot = p.dB * p.area
            if  unshot > maxUnshot:
                maxUnshot = unshot
                self.maxUnshotIndex = p.idt
        print(f"[initRadiosity] maxUnshotIndex = {self.maxUnshotIndex}")

    def solveRadiosity(self, epsilon=1., n=100):
        step = 0.
        sumdRad = epsilon
        while sumdRad >= epsilon and step < n :
            src = self.get(self.maxUnshotIndex)
            maxUnshot = 0.
            sumdRad = 0.
            for rec in self.poly:
                if rec.idt != src.idt:
                    dRad = src.dB * rec.rho * rec.fij[src.idt]
                    rec.dB += dRad
                    rec.B += dRad
                    tmp_unshot = rec.dB * rec.area
                    sumdRad += dRad
                    if tmp_unshot > maxUnshot:
                        self.maxUnshotIndex = rec.idt
                        maxUnshot = tmp_unshot
            print(f"[{step}] maxUnshotIndex={self.maxUnshotIndex} dB={src.dB} sum dRad = {sumdRad}")
            src.dB = 0
            step += 1

    def showAllBi(self):
        for p in self.poly:
            print(f"B{p.idt} = {p.B}")


    def initFormFactors(self):
        size = len(self.poly)
        for p in self.poly:
            p.initFormFactors(len(self.poly))

    def getFormFactors_uni_sq(self):
        for poly_i in self.poly:
            for poly_j in self.poly:
                if poly_i.idt != poly_j.idt:
                    poly_i.fij[poly_j.idt] = self.getFormFactor_uni_sq(poly_i, poly_j)

    def getFormFactors_uni_sq2(self):
        count = len(self.poly)
        for i in range(count-1):
            poly_i = self.poly[i]
            for j in range(i+1, count):
                poly_j = self.poly[j]
                fij = self.getFormFactor_uni_sq(poly_i, poly_j)
                poly_i.fij[poly_j.idt] = fij
                poly_j.fij[poly_i.idt] = fij * poly_i.area / poly_j.area
                    
    def getFormFactors_rnd(self, n):
        for poly_i in self.poly:
            for poly_j in self.poly:
                if poly_i.idt != poly_j.idt:
                    fij = self.getFormFactor_rnd(poly_i, poly_j, n)
                    #print(f"F({poly_i.idt}, {poly_j.idt}) = {fij}")
                    # TODO: if poly_i != 0, poly_i.fij = (poly_i.fij + fij)/2 ; if poly_j.fij != 0, poly_j.fij = poly_i.fij (??)
                    # TODO: to garanty reciprocity !!! ONLY if i an d j are of the same form !
                    #tmp_fij = poly_i.fij[poly_j.idt]
                    #if tmp_fij != 0.:
                    #    fij = (tmp_fij + fij)/2.
                    poly_i.fij[poly_j.idt] = fij
                    #poly_j.fij[poly_i.idt] = fij
                    #if (poly_i.idt == 24 and poly_j.idt == 0) or (poly_j.idt == 24 and poly_i.idt == 0):
                    #   print(f"[{poly_i.idt} -> {poly_j.idt}] Fij = {poly_i.fij[poly_j.idt]}")
                    #   print(f"[{poly_j.idt} -> {poly_i.idt}] Fij = {poly_j.fij[poly_i.idt]}")

    def getFormFactor_uni_sq(self, ri, rj):
        fij = 0.
        aj = rj.area
        patchs_i = ri.getPatchs()
        patchs_j = rj.getPatchs()
        li = len(patchs_i)
        lj = len(patchs_j)
        samples = li * lj

        for xi in patchs_i:
            for xj in patchs_j:
                # get ray from r1 to r2
                rij = xj - xi
                r = np.linalg.norm(rij)
                nrij = rij / r
                r2 = r*r
                costhetai = np.dot(ri.norm, nrij)
                costhetaj = -1.*np.dot(rj.norm, nrij)
                deltaf = ((costhetai * costhetaj) / (np.pi * r2 + aj/lj))
                #if ri.idt == 1 and (rj.idt == 24 or rj.idt == 27):
                #    if (xi[0]==1.25 and xi[1] == 0.25 and xj[1] == 0.25 and xj[2] == 0.25 and xj[0] == 0.) or (xi[0]==1.75 and xi[1] == 0.25 and xj[1] == 0.25 and xj[2] == 0.25 and xj[0] == 2.):
                #        print(f"{xi} -> {xj} : deltaf: {deltaf} costhetai: {costhetai}, costhetaj: {costhetaj} r2: {r2}")
                if (deltaf > 0.):
                    fij += deltaf
        fij = fij * aj/samples
        return fij


    def getFormFactor_rnd(self, ri, rj, n=1):
        fij = 0.
        aj = rj.area
        for i in range(n):
            # get a point on r1 and r2
            xi = ri.getRandomPoint()
            xj = rj.getRandomPoint()
            #print(f"xi: {xi} xj: {xj}")
            # get ray from r1 to r2
            rij = xj - xi
            r = np.linalg.norm(rij)
            nrij = rij / r
            #print(f"normalized ray vector: {nrij} ({nrij[0]*nrij[0] + nrij[1]*nrij[1] + nrij[2]*nrij[2]})")
            r2 = r*r
            costhetai = np.dot(ri.norm, nrij)
            costhetaj = -1.*np.dot(rj.norm, nrij)
            #print(f"costhetai: {costhetai}, costhetaj: {costhetaj} r2: {r2}")
            deltaf = ((costhetai * costhetaj) / (np.pi * r2 + aj/n))
            #print(f"deltaf: {deltaf}")
            if (deltaf > 0.):
                fij += deltaf
        fij = fij * aj/n
        return fij

    def showFij(self, i):
        self.poly[i].dumpFij()

print("class Mesh: OK")

class Mesh: OK


## Les calculs analitiques

- pas de changement

In [6]:
def getThFij(a, b, c):
    X = a/c
    Y = b/c
    th_fij = 2/(np.pi*X*Y) * ( np.log(np.power((1+X*X)*(1+Y*Y) / (1+X*X+Y*Y), 1/2  ))
        + X*np.sqrt(1+Y*Y)*np.arctan(X/np.sqrt(1+Y*Y))
        + Y*np.sqrt(1+X*X)*np.arctan(Y/np.sqrt(1+X*X))
        - X*np.arctan(X) - Y*np.arctan(Y))
    print(f"theoritical para. ({a},{b},{c}) Fij = {th_fij}")

def getThPerpFij(a, b, c):
    H = a/c
    W = b/c
    th_fij = (1./(np.pi*W)) * (W*np.arctan(1./W) + H*np.arctan(1./H) - np.sqrt(H*H+W*W)*np.arctan(1./np.sqrt(H*H+W*W))
                                + (1./4.) * np.log( ((1+W*W)*(1+H*H) / (1+W*W+H*H))
                                                * np.power((W*W*(1+W*W+H*H) / ((1+W*W)*(W*W+H*H))),W*W)
                                                * np.power((H*H*(1+W*W+H*H) / ((1+H*H)*(H*H+W*W))),H*H)
                                                ))
    print(f"theoritical perp. ({a},{b},{c}) Fij = {th_fij}")

def g_para_z(x, y, eta, xi, z):
    res = (1./(2.*np.pi)) * (
            (y - eta)*np.sqrt( (x-xi)*(x-xi) + z*z ) * np.arctan( (y-eta)/np.sqrt((x-xi)*(x-xi) + z*z ) )
            + (x-xi)*np.sqrt(  (y-eta)*(y-eta) + z*z ) * np.arctan( (x-xi)/np.sqrt((y-eta)*(y-eta) + z*z ) )
            - (z*z/2.)*np.log( (x-xi)*(x-xi) + (y-eta)*(y-eta) + z*z )
            )
    return res

def getThFij_para_z(r1, r2):
    # http://www.thermalradiation.net/sectionc/C-13.html
    z = r2.cmax[2] - r1.cmax[2]
    x = np.array([r1.cmin[0], r1.cmax[0]])
    y = np.array([r1.cmin[1], r1.cmax[1]])
    xi = np.array([r2.cmin[0], r2.cmax[0]])
    eta = np.array([r2.cmin[1], r2.cmax[1]])

    f12 = 0.
    for l in range(1,3):
        for k in range(1,3):
            for j in range(1,3):
                for i in range(1,3):
                    f12 += np.power(-1, i+j+k+l)*g_para_z(x[i-1], y[j-1], eta[k-1], xi[l-1], z)
    f12 *= 1./ ( (x[2-1] - x[1-1])*(y[2-1] - y[1-1]) )
    print(f"Th Fij ({r1.idt} -> {r2.idt}) = {f12}")

def g_perp_x(x, y, eta, xi):
    K = (y -eta) / np.sqrt(x*x + xi*xi)
    res = (1./(2.*np.pi)) * (
            (y - eta)*np.sqrt( x*x + xi*xi ) * np.arctan(K) - (1./4.)*(x*x +xi*xi)*(1-K*K)
            * np.log((x*x + xi*xi)*(1+K*K))
            )
    return res

def getThFij_perp_x(r1, r2):
    # http://www.thermalradiation.net/sectionc/C-15.html
    if r1.cmin[0] == r2.cmin[2]:
        print("common edge case")
        return 0.
    x = np.array([r1.cmin[0], r1.cmax[0]])
    y = np.array([r1.cmin[1], r1.cmax[1]])
    xi = np.array([r2.cmin[2], r2.cmax[2]])
    eta = np.array([r2.cmin[1], r2.cmax[1]])

    f12 = 0.
    for l in range(1,3):
        for k in range(1,3):
            for j in range(1,3):
                for i in range(1,3):
                    f12 += np.power(-1, i+j+k+l)*g_perp_x(x[i-1], y[j-1], eta[k-1], xi[l-1])
    f12 *= 1./ ( (x[2-1] - x[1-1])*(y[2-1] - y[1-1]) )
    print(f"Th Fij perp ({r1.idt} -> {r2.idt}) = {f12}")

print("analytic functions: OK")

analytic functions: OK


## Initialisation du maillage

In [7]:
c = 2.

mesh = Mesh()

nz_up = np.array([0., 0., 1.])
nz_down = np.array([0., 0., -1.])

ny_up = np.array([0., 1., 0.])
ny_down = np.array([0., -1., 0.])

nx_up = np.array([1., 0., 0.])
nx_down = np.array([-1., 0., 0.])
print("normal vectors: OK")

normal vectors: OK


In [8]:
WIDTH = 10
HEIGHT = 10
Rect.count = -1
# Construct Mesh
## 1. Floor
print("## FLOOR ##")
for y in range(HEIGHT):
    for x in range(WIDTH):
        mesh.add(Rect(np.array([x, y, 0.]), np.array([x+1., y+1., 0.]), nz_up))

## 1. Ceiling
print("## CEILING ##")
for y in range(HEIGHT):
    for x in range(WIDTH):
        mesh.add(Rect(np.array([x, y, c]), np.array([x+1., y+1., c]), nz_down))

# North Wall
print("## NORTH WALL ##")
y = 0
for x in range(WIDTH):
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x+1, y, c]), ny_up))

# South Wall
print("## SOUTH WALL ##")
y = HEIGHT
for x in range(WIDTH):
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x+1, y, c]), ny_down))

# West Wall
print("## WEST WALL ##")
x = 0
for y in range(HEIGHT):
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x, y+1, c]), nx_up))

# East Wall
print("## EAST WALL ##")
x = WIDTH
for y in range(HEIGHT):
    mesh.add(Rect(np.array([x, y, 0.]), np.array([x, y+1, c]), nx_down))
print("mesh: OK")

## FLOOR ##
0 [0. 0. 0.] [1. 1. 0.] area:1.0
1 [1. 0. 0.] [2. 1. 0.] area:1.0
2 [2. 0. 0.] [3. 1. 0.] area:1.0
3 [3. 0. 0.] [4. 1. 0.] area:1.0
4 [4. 0. 0.] [5. 1. 0.] area:1.0
5 [5. 0. 0.] [6. 1. 0.] area:1.0
6 [6. 0. 0.] [7. 1. 0.] area:1.0
7 [7. 0. 0.] [8. 1. 0.] area:1.0
8 [8. 0. 0.] [9. 1. 0.] area:1.0
9 [9. 0. 0.] [10.  1.  0.] area:1.0
10 [0. 1. 0.] [1. 2. 0.] area:1.0
11 [1. 1. 0.] [2. 2. 0.] area:1.0
12 [2. 1. 0.] [3. 2. 0.] area:1.0
13 [3. 1. 0.] [4. 2. 0.] area:1.0
14 [4. 1. 0.] [5. 2. 0.] area:1.0
15 [5. 1. 0.] [6. 2. 0.] area:1.0
16 [6. 1. 0.] [7. 2. 0.] area:1.0
17 [7. 1. 0.] [8. 2. 0.] area:1.0
18 [8. 1. 0.] [9. 2. 0.] area:1.0
19 [9. 1. 0.] [10.  2.  0.] area:1.0
20 [0. 2. 0.] [1. 3. 0.] area:1.0
21 [1. 2. 0.] [2. 3. 0.] area:1.0
22 [2. 2. 0.] [3. 3. 0.] area:1.0
23 [3. 2. 0.] [4. 3. 0.] area:1.0
24 [4. 2. 0.] [5. 3. 0.] area:1.0
25 [5. 2. 0.] [6. 3. 0.] area:1.0
26 [6. 2. 0.] [7. 3. 0.] area:1.0
27 [7. 2. 0.] [8. 3. 0.] area:1.0
28 [8. 2. 0.] [9. 3. 0.] area:1.0
29 [9.

234 [10.  4.  0.] [10.  5.  2.] area:2.0
235 [10.  5.  0.] [10.  6.  2.] area:2.0
236 [10.  6.  0.] [10.  7.  2.] area:2.0
237 [10.  7.  0.] [10.  8.  2.] area:2.0
238 [10.  8.  0.] [10.  9.  2.] area:2.0
239 [10.  9.  0.] [10. 10.  2.] area:2.0
mesh: OK


## Recherche des facteurs de forme

In [9]:
mesh.initFormFactors()
mesh.getFormFactors_uni_sq2()
print("Form Factors: OK")
mesh.showFij(1)

Form Factors: OK
[1 -> 0] Fij = 0.0
[1 -> 1] Fij = 0.0
[1 -> 2] Fij = 0.0
[1 -> 3] Fij = 0.0
[1 -> 4] Fij = 0.0
[1 -> 5] Fij = 0.0
[1 -> 6] Fij = 0.0
[1 -> 7] Fij = 0.0
[1 -> 8] Fij = 0.0
[1 -> 9] Fij = 0.0
[1 -> 10] Fij = 0.0
[1 -> 11] Fij = 0.0
[1 -> 12] Fij = 0.0
[1 -> 13] Fij = 0.0
[1 -> 14] Fij = 0.0
[1 -> 15] Fij = 0.0
[1 -> 16] Fij = 0.0
[1 -> 17] Fij = 0.0
[1 -> 18] Fij = 0.0
[1 -> 19] Fij = 0.0
[1 -> 20] Fij = 0.0
[1 -> 21] Fij = 0.0
[1 -> 22] Fij = 0.0
[1 -> 23] Fij = 0.0
[1 -> 24] Fij = 0.0
[1 -> 25] Fij = 0.0
[1 -> 26] Fij = 0.0
[1 -> 27] Fij = 0.0
[1 -> 28] Fij = 0.0
[1 -> 29] Fij = 0.0
[1 -> 30] Fij = 0.0
[1 -> 31] Fij = 0.0
[1 -> 32] Fij = 0.0
[1 -> 33] Fij = 0.0
[1 -> 34] Fij = 0.0
[1 -> 35] Fij = 0.0
[1 -> 36] Fij = 0.0
[1 -> 37] Fij = 0.0
[1 -> 38] Fij = 0.0
[1 -> 39] Fij = 0.0
[1 -> 40] Fij = 0.0
[1 -> 41] Fij = 0.0
[1 -> 42] Fij = 0.0
[1 -> 43] Fij = 0.0
[1 -> 44] Fij = 0.0
[1 -> 45] Fij = 0.0
[1 -> 46] Fij = 0.0
[1 -> 47] Fij = 0.0
[1 -> 48] Fij = 0.0
[1 -> 49] Fij

## Source Lumineuse

In [10]:
mesh.resetLights()

light = mesh.get(113) 
#light = mesh.get(index) 
#light.initLightSource(31.830, 0.)
# Examples in cd/m^2
# Office (Accounting) 538
# Office              323
# Operating Room    19375
# Twilight             11
# Deep Twilight         1.07
# Moonlight             0.04
# Dusk                107
# Overcast day       1076
# Full dayligh      10764
# Direct sunlight  107639
light.initLightSource(538, 0.)
#light = mesh.get(112)
#light.initLightSource(500, 0.)
print("light source(s) : OK")

light source(s) : OK


## Radiosité

Où on teste enfin le système...

In [11]:
mesh.resetRadiosity()
mesh.initRadiosity()
mesh.solveRadiosity(1., 500)
mesh.showAllBi()

[initRadiosity] maxUnshotIndex = 113
[0.0] maxUnshotIndex=13 dB=538 sum dRad = 254.35676992856563
[1.0] maxUnshotIndex=203 dB=20.579768857521387 sum dRad = 9.729746342891504
[2.0] maxUnshotIndex=3 dB=10.090029458544064 sum dRad = 9.94705835661102
[3.0] maxUnshotIndex=12 dB=15.45431766332692 sum dRad = 6.527790899183865
[4.0] maxUnshotIndex=14 dB=14.441247796253919 sum dRad = 6.713792925459972
[5.0] maxUnshotIndex=23 dB=14.441247796253919 sum dRad = 6.869738365050729
[6.0] maxUnshotIndex=204 dB=14.348486551753599 sum dRad = 7.105583277718432
[7.0] maxUnshotIndex=202 dB=7.008320904492894 sum dRad = 6.944530219925686
[8.0] maxUnshotIndex=4 dB=7.008320904492893 sum dRad = 6.820365056873014
[9.0] maxUnshotIndex=2 dB=11.491481861202967 sum dRad = 4.880771541184066
[10.0] maxUnshotIndex=24 dB=11.491481861202965 sum dRad = 4.782659631120592
[11.0] maxUnshotIndex=22 dB=10.535472604118306 sum dRad = 5.2525308099789
[12.0] maxUnshotIndex=15 dB=10.535472604118304 sum dRad = 5.1225920737073585
[13.

In [12]:
B_floor = np.zeros([HEIGHT, WIDTH])
B_ceiling = np.zeros([HEIGHT, WIDTH])
B_NWall = np.zeros([WIDTH])
B_SWall = np.zeros([WIDTH])
B_EWall = np.zeros([HEIGHT])
B_WWall = np.zeros([HEIGHT])

def polyToArrays():
    floorSize = (HEIGHT*WIDTH)
    idxCeiling = floorSize
    idxNWall = 2*floorSize
    idxSWall = idxNWall + WIDTH
    idxEWall = idxSWall + WIDTH
    idxWWall = idxEWall + HEIGHT
    endWWall = idxWWall + HEIGHT
    
    for p in mesh.poly:
        if p.idt >= 0 and p.idt < idxCeiling:
            x = p.idt % WIDTH
            y = int(p.idt / WIDTH)
            B_floor[y, x] = p.B
        elif p.idt >= idxCeiling and p.idt < idxNWall:
            x = (p.idt - idxCeiling) % WIDTH
            y = int((p.idt - idxCeiling) / WIDTH)            
            B_ceiling[y, x] = p.B
        elif p.idt >= idxNWall and p.idt < idxSWall:
            x = (p.idt - idxNWall)
            B_NWall[x] = p.B
        elif p.idt >= idxSWall and p.idt < idxEWall:
            x = (p.idt - idxSWall)
            B_SWall[x] = p.B
        elif p.idt >= idxEWall and p.idt < idxWWall:
            y = (p.idt - idxEWall)
            B_WWall[y] = p.B
        elif p.idt >= idxWWall and p.idt < endWWall:
            y = (p.idt - idxWWall)
            B_EWall[y] = p.B

def fdisplay(data, msg=""):
    df = pd.DataFrame(data).sort_index(ascending=False, axis=0)
    if msg!="":
        display(HTML("<b>"+msg+"</b>"))
    display(HTML(df.style.render()+"</br>"))

def fdisplayTrans(data, msg=""):
    df = pd.DataFrame(data).sort_index(ascending=True, axis=0).transpose()
    if msg!="":
        display(HTML("<b>"+msg+"</b>"))
    display(HTML(df.style.render()+"</br>"))
    
polyToArrays()
#pd.DataFrame(B_NWall).transpose()
fdisplay(B_floor, "Floor")
fdisplay(B_ceiling, "Ceiling")
fdisplayTrans(B_SWall, "South Wall (!Watch out, y axis reverse)")
fdisplayTrans(B_NWall, "North Wall(!Watch out, y axis reverse)")
fdisplay(B_WWall, "West Wall")
fdisplay(B_EWall, "East Wall")



,0,1,2,3,4,5,6,7,8,9
9,0.0867347,0.0998997,0.108981,0.112189,0.108991,0.100304,0.0880697,0.0744568,0.0612038,0.0493574
8,0.132136,0.157596,0.175492,0.181623,0.174677,0.156962,0.133203,0.108229,0.0853657,0.0661449
7,0.210248,0.263124,0.301321,0.314291,0.29805,0.258919,0.209596,0.161512,0.120851,0.0891784
6,0.352003,0.46981,0.559034,0.59078,0.549244,0.454071,0.343762,0.246683,0.172638,0.120132
5,0.634593,0.899871,1.13245,1.22681,1.10823,0.850361,0.584977,0.381837,0.245998,0.159904
4,1.23446,1.79175,2.49043,2.82372,2.44212,1.67726,1.01162,0.585937,0.34277,0.206873
3,2.05023,3.48079,5.67654,6.93378,5.59757,3.28967,1.68908,0.856193,0.453214,0.254468
2,2.99878,5.93672,11.6697,15.6659,11.559,5.66633,2.48884,1.12388,0.547381,0.290077
1,3.52621,7.50549,16.1241,22.6521,15.9905,7.17959,2.93384,1.24941,0.580544,0.298727
0,3.15596,6.49955,12.8534,17.274,12.7273,6.13534,2.64817,1.11075,0.520092,0.272518


,0,1,2,3,4,5,6,7,8,9
9,0.0456194,0.0534097,0.0587728,0.0606553,0.0588106,0.0538684,0.0470286,0.0395525,0.0323848,0.026052
8,0.0705206,0.0860491,0.0968834,0.100458,0.0961133,0.085504,0.0717118,0.0576446,0.0450868,0.0347263
7,0.115407,0.149173,0.173335,0.18111,0.170131,0.145109,0.114966,0.0869078,0.0640616,0.046747
6,0.202082,0.280831,0.33963,0.359465,0.329914,0.265293,0.194197,0.135289,0.0924762,0.0631756
5,0.390243,0.561262,0.70411,0.757533,0.679979,0.512042,0.341268,0.215222,0.134223,0.0848171
4,0.820381,1.08317,1.38658,1.50841,1.3393,0.96994,0.599278,0.341308,0.192068,0.111205
3,1.33181,1.83041,2.38274,2.6142,2.30963,1.64757,0.978892,0.514958,0.261892,0.1387
2,1.85231,2.60449,3.47017,3.8521,3.37544,2.36329,1.37433,0.688123,0.322891,0.158742
1,2.11889,3.0771,4.22447,542.752,4.11433,2.79876,1.58229,0.761958,0.339405,0.160029
0,1.92453,3.0029,4.42607,5.20883,4.31935,2.67455,1.45452,0.644634,0.284187,0.136565


,0,1,2,3,4,5,6,7,8,9
0,0.301219,0.342959,0.371333,0.381672,0.372905,0.34768,0.311211,0.269412,0.227299,0.188241


,0,1,2,3,4,5,6,7,8,9
0,2.13815,4.2396,8.91067,12.9161,8.81879,4.00911,1.69356,0.683629,0.309578,0.159387


,0
9,0.125269
8,0.18404
7,0.281091
6,0.446867
5,0.73508
4,1.22659
3,1.99001
2,2.96878
1,3.52036
0,3.02188


,0
9,0.14402
8,0.185092
7,0.238642
6,0.306785
5,0.389396
4,0.480918
3,0.566388
2,0.620175
1,0.613865
0,0.535493


## Représentation

- une représentation rapide avec une dataframe

Convertion basique des valeurs de radiosité en normalisant les valeurs sur la plage correspondant au sol à partir de la radiosité maximale de la surface). Quick'n dirty

In [13]:
lighting_color = np.zeros([HEIGHT, WIDTH])

def normalizeMatrix(data, m=0):
    maxB = data.max() if m == 0 else m
    idx = 1 if len(np.shape(data)) == 2 else 0
    d = np.full([np.shape(data)[idx]], maxB)
    data /=  d
    return maxB

data = np.copy(B_floor)
normalizeMatrix(data)
convertRatioToRGB(data)
fakeDisplay("testing...", False)

,0,1,2,3,4,5,6,7,8,9
9,0,1,1,1,1,1,0,0,0,0
8,1,1,1,2,1,1,1,1,0,0
7,2,2,3,3,3,2,2,1,1,1
6,3,5,6,6,6,5,3,2,1,1
5,7,10,12,13,12,9,6,4,2,1
4,13,20,28,31,27,18,11,6,3,2
3,23,39,63,78,63,37,19,9,5,2
2,33,66,131,176,130,63,28,12,6,3
1,39,84,181,255,180,80,33,14,6,3
0,35,73,144,194,143,69,29,12,5,3


In [14]:
def destructiveGnlRatioToRGB(source):
    #normalizeMatrix(source)
    dim = len(np.shape(source))
    if dim == 1:
        maxu = np.shape(source)[0]
        for u in range(maxu):
            if source[u] != 0.:
                source[u] = int(abs(source[u]) * 255)
    elif dim == 2:
        maxy = np.shape(source)[0]
        maxx = np.shape(source)[1]
        for y in range(maxy):
            for x in range(maxx):
                if source[y, x] != 0.:
                    source[y, x] = int(abs(source[y, x]) * 255)

def fcoldisplay(data, msg=""):
    df = pd.DataFrame(data).sort_index(ascending=False, axis=0)
    s = df.style.applymap(color_max_red)
    if msg!="":
        display(HTML("<b>"+msg+"</b>"))
    display(HTML(s.render()+"</br>"))

def fcoldisplayTrans(data, msg=""):
    df = pd.DataFrame(data).sort_index(ascending=True, axis=0).transpose()
    s = df.style.applymap(color_max_red)
    if msg!="":
        display(HTML("<b>"+msg+"</b>"))
    display(HTML(s.render()+"</br>"))

B_floor_std = np.copy(B_floor)
m = normalizeMatrix(B_floor_std)
destructiveGnlRatioToRGB(B_floor_std)

#B_ceiling_std = np.copy(B_ceiling)
#normalizeMatrix(B_ceiling_std, m)
#destructiveGnlRatioToRGB(B_ceiling_std)

B_NWall_std = np.copy(B_NWall)
normalizeMatrix(B_NWall_std, m)
destructiveGnlRatioToRGB(B_NWall_std)

B_SWall_std = np.copy(B_SWall)
normalizeMatrix(B_SWall_std, m)
destructiveGnlRatioToRGB(B_SWall_std)

#B_WWall_std = np.copy(B_WWall)
#normalizeMatrix(B_WWall_std, m)
#destructiveGnlRatioToRGB(B_WWall_std)

#B_EWall_std = np.copy(B_EWall)
#normalizeMatrix(B_EWall_std, m)
#destructiveGnlRatioToRGB(B_EWall_std)

fcoldisplayTrans(B_SWall_std)
fcoldisplay(B_floor_std)
fcoldisplayTrans(B_NWall_std)

,0,1,2,3,4,5,6,7,8,9
0,3,3,4,4,4,3,3,3,2,2


,0,1,2,3,4,5,6,7,8,9
9,0,1,1,1,1,1,0,0,0,0
8,1,1,1,2,1,1,1,1,0,0
7,2,2,3,3,3,2,2,1,1,1
6,3,5,6,6,6,5,3,2,1,1
5,7,10,12,13,12,9,6,4,2,1
4,13,20,28,31,27,18,11,6,3,2
3,23,39,63,78,63,37,19,9,5,2
2,33,66,131,176,130,63,28,12,6,3
1,39,84,181,255,180,80,33,14,6,3
0,35,73,144,194,143,69,29,12,5,3


,0,1,2,3,4,5,6,7,8,9
0,24,47,100,145,99,45,19,7,3,1


## Bilan

### Général

- on a bien le jeu de réfexion autour de la lumière
- sans surprise, la détermination des facteurs de formes de la scène font très mal (126 surfaces x 4 patchs par surface avec un algorithme en O(n^2)
- qui ne sont calculés qu'une fois tant que la géométrie de la scène n'est pas changée
- pré-calculs... qui doivent être fait à la génération du vaisseau
- ... mais aucune optimisation n'a été mise en place
    - coté symétrie, on peut lier Fji à Fij en fonction des aires
    - passer de 16 rayons à 4
    - ...

### Affichage

Il y a un problème important avec la technique que nous avons utilisée jusque là pour convertir la radiance en niveaux de gris : **quelque soit la puissance de la source lumineuse, toutes les valeurs sont ramenées dans la plage [0 ; 1] (puis [0; 255] de manière linéaire**. 

## Et maintenant ?

plusieurs pistes :

- améliorer le calcul des facteurs de forme
- introduire les occlusions
- corriger la répartition des niveaux de gris
- "tonal mode" pour simuler le comportement de l'oeil humain et gérer les écarts de luminosité

## Tone mapping operators

- **TODO:** Attention à la somme pour la moyenne
- **TODO:** Attention au **gamma**, si gamma = 0.45 (1/2.2), le résultat ressemble plus à ce qui est attendu quand la source est faible

In [15]:
mesh.resetLights()

light = mesh.get(113) 
#light = mesh.get(index) 
#light.initLightSource(31.830, 0.)
# Examples in cd/m^2
# Office (Accounting) 538
# Office              323
# Operating Room    19375
# Twilight             11
# Deep Twilight         1.07
# Moonlight             0.04
# Dusk                107
# Overcast day       1076
# Full dayligh      10764
# Direct sunlight  107639
intensity = 323
light.initLightSource(intensity, 0.)
print("light source(s) : OK")

mesh.resetRadiosity()
mesh.initRadiosity()
mesh.solveRadiosity(intensity/1000, 500)
print("Radiance : OK")

polyToArrays()
print("Poly to arrays : OK")

light source(s) : OK
[initRadiosity] maxUnshotIndex = 113
[0.0] maxUnshotIndex=13 dB=323 sum dRad = 152.70861837718718
[1.0] maxUnshotIndex=203 dB=12.355511786207076 sum dRad = 5.841464811810335
[2.0] maxUnshotIndex=3 dB=6.0577686154456 sum dRad = 5.971932805177247
[3.0] maxUnshotIndex=12 dB=9.278335697499248 sum dRad = 3.919101227576928
[4.0] maxUnshotIndex=14 dB=8.670117171356905 sum dRad = 4.030771589077272
[5.0] maxUnshotIndex=23 dB=8.670117171356905 sum dRad = 4.1243968251141006
[6.0] maxUnshotIndex=204 dB=8.614425940922697 sum dRad = 4.265991447403445
[7.0] maxUnshotIndex=202 dB=4.207597866452054 sum dRad = 4.1692997417025985
[8.0] maxUnshotIndex=4 dB=4.207597866452053 sum dRad = 4.094754485817808
[9.0] maxUnshotIndex=2 dB=6.8991610430642325 sum dRad = 2.9302773379227762
[10.0] maxUnshotIndex=24 dB=6.899161043064231 sum dRad = 2.8713737190556716
[11.0] maxUnshotIndex=22 dB=6.325200095037569 sum dRad = 3.153471099671343
[12.0] maxUnshotIndex=15 dB=6.325200095037567 sum dRad = 3.07

### Tone Reproduction for Realistic Computer Generated Images, J.E. Tumblin & H.E Rushmeier

#### Version originale 1991

In [16]:
# https://smartech.gatech.edu/bitstream/handle/1853/3699/91-13.pdf?sequence=1&isAllowed=y
# Tone Reproduction for Realistic Computer Generated Images, J.E. Tumblin, H.E. Rushmeier 1991
gamma = 2.2

avg_llrw = 0.
count = 0
for p in mesh.poly:
    if p.B != 0. and p.rho != -1. :
        avg_llrw += np.log10(p.B * np.pi / 10000)
        count += 1

avg_llrw /= count
llwrw = avg_llrw + 0.84

ldmax = 150.*np.pi / 10000

llwd = -1.569

arw = 0.4*llwrw + 2.92
brw = -0.4*(llwrw*llwrw) + (-2.584*llwrw) + 2.0208

ad = 0.4*llwd + 2.92
bd = -0.4*(llwd*llwd) + (-2.584*llwd) + 2.0208

print(f"ldmax: {ldmax} ; llwrw: {llwrw} (avg_llrw: {avg_llrw}) ; llwd: {llwd} ; arw: {arw} ; brw: {brw} ; ad: {ad} ; bd: {bd}")

# Power law
# np.power( (1/ldmax)* np.power((B_floor[y, x]*np.pi/10000), arw/ad ) * np.power(10., (brw - bd)/ad), 1/gamma)
#  = k*(B_floor[y, x]*np.pi/10000)^q
k = np.power((1/ldmax), (1/gamma)) * np.power(10, (((brw - bd)/ad)*(1/gamma)))
q = (arw/ad) * 1/gamma
#print(f"k: {k} ; q: {q}")
#k*np.power(12.5688*np.pi/10000, q)


B_floor_sf = np.zeros([HEIGHT, WIDTH])
for y in range(HEIGHT):
    for x in range(WIDTH):
        #n = np.power( (1/ldmax)* np.power((B_floor[y, x]*np.pi/10000), arw/ad ) * np.power(10., (brw - bd)/ad), 1/gamma)
        n = k*np.power(B_floor[y, x]*np.pi/10000, q)
        B_floor_sf[y, x] = n 

fdisplay(B_floor_sf, f"Floor sf")


data = np.copy(B_floor_sf)
lighting_color = np.zeros([HEIGHT, WIDTH])
convertRatioToRGB(data)
fakeDisplay("testing...", True)
k = np.power((1/ldmax), (1/gamma)) * np.power(10, (((brw - bd)/ad)*(1/gamma)))
q = (arw/ad) * 1/gamma
print(f"k: {k} ; q: {q}")
k*np.power(12.5688*np.pi/10000, q)

ldmax: 0.047123889803846894 ; llwrw: -3.0686817820564345 (avg_llrw: -3.9086817820564344) ; llwd: -1.569 ; arw: 1.692527287177426 ; brw: 6.183550573023805 ; ad: 2.2923999999999998 ; bd: 5.0903916


,0,1,2,3,4,5,6,7,8,9
9,0.168424,0.177054,0.182361,0.184005,0.182033,0.176849,0.169149,0.159756,0.149436,0.138807
8,0.194883,0.207391,0.214847,0.216923,0.213696,0.205834,0.194569,0.181313,0.167257,0.153226
7,0.229708,0.248352,0.258996,0.261748,0.256385,0.243974,0.226887,0.2077,0.188271,0.16958
6,0.278709,0.304926,0.320474,0.324735,0.315737,0.295273,0.268352,0.239935,0.212863,0.187873
5,0.349493,0.380426,0.406015,0.414353,0.399308,0.364601,0.321184,0.278608,0.241178,0.208151
4,0.421325,0.473591,0.521402,0.540211,0.514555,0.455242,0.385767,0.322701,0.272651,0.232977
3,0.49799,0.581323,0.67508,0.718594,0.66994,0.564797,0.45669,0.367517,0.303108,0.268921
2,0.558514,0.68611,0.852089,0.93844,0.848573,0.672596,0.518032,0.403225,0.324187,0.282975
1,0.585648,0.738346,0.947448,1.0606,0.944635,0.726473,0.546501,0.418452,0.329965,0.28186
0,0.56634,0.702845,0.877934,0.969559,0.875233,0.691723,0.527582,0.40764,0.315443,0.254377


,0,1,2,3,4,5,6,7,8,9
9,42,45,46,46,46,45,43,40,38,35
8,49,52,54,55,54,52,49,46,42,39
7,58,63,66,66,65,62,57,52,48,43
6,71,77,81,82,80,75,68,61,54,47
5,89,97,103,105,101,92,81,71,61,53
4,107,120,132,137,131,116,98,82,69,59
3,126,148,172,183,170,144,116,93,77,68
2,142,174,217,239,216,171,132,102,82,72
1,149,188,241,270,240,185,139,106,84,71
0,144,179,223,247,223,176,134,103,80,64


k: 6.604314879836959 ; q: 0.3356004995117118


1.0308442968251996

#### Avec des corrections

- Efficient Radiosity Simulation using Perceptual Metrics and Parallel Processing, Simon Gibson, Thèse, (1999)
- Tone Reproduction and Physically Based Spectral Rendering, K. Devlin, A. Chalmers, A. Wilkie & W. Purgathofer (2002)

In [17]:
# https://smartech.gatech.edu/bitstream/handle/1853/3699/91-13.pdf?sequence=1&isAllowed=y
# Tone Reproduction for Realistic Computer Generated Images, J.E. Tumblin, H.E. Rushmeier 1991
# http://apt.cs.manchester.ac.uk/ftp/pub/TR/UMCS-99-9-1.ps.Z lamberts -> cd/m^2 , coeff. (p57)
# cf. # Correction in http://doc.gold.ac.uk/~mas01kd/publications/egSTAR.pdf (4.2 Spatially uniform operators) (lamberts to cd/m^-2 ; fix 2nd coeff. for beta)
gamma = 2.2

avg_llrw = 0.
count = 0
for p in mesh.poly:
    if p.B != 0. and p.rho != -1. :
        avg_llrw += np.log10(p.B)
        count += 1

avg_llrw /= count
llwrw = avg_llrw + 0.84

ldmax = 150.

# display : (lamberts -> cd/m^2)
llwd = 1.929419

# Watch out: -0.218 / +0.218 ?
arw = 0.4*llwrw + 1.519
brw = -0.4*(llwrw*llwrw) - 0.218*llwrw + 6.1642

ad = 0.4*llwd + 1.519
bd = -0.4*(llwd*llwd) - 0.218*llwd + 6.1642

print(f"ldmax: {ldmax} ; llwrw: {llwrw} (avg_llrw: {avg_llrw}) ; llwd: {llwd} ; arw: {arw} ; brw: {brw} ; ad: {ad} ; bd: {bd}")

# Power law
# np.power( (1/ldmax)* np.power((B_floor[y, x]), arw/ad ) * np.power(10., (brw - bd)/ad), 1/gamma)
#  = k*(B_floor[y, x]*np.pi/10000)^q
k = np.power((1/ldmax), (1/gamma)) * np.power(10, (((brw - bd)/ad)*(1/gamma)))
q = (arw/ad) * 1/gamma


B_floor_sf = np.zeros([HEIGHT, WIDTH])
for y in range(HEIGHT):
    for x in range(WIDTH):
        #n = np.power( (1/ldmax)* np.power((B_floor[y, x]*np.pi/10000), arw/ad ) * np.power(10., (brw - bd)/ad), 1/gamma)
        n = k*np.power(B_floor[y, x], q)
        B_floor_sf[y, x] = n 

fdisplay(B_floor_sf, f"Floor sf")


data = np.copy(B_floor_sf)
lighting_color = np.zeros([HEIGHT, WIDTH])
convertRatioToRGB(data)
fakeDisplay("testing...", True)
k = np.power((1/ldmax), (1/gamma)) * np.power(10, (((brw - bd)/ad)*(1/gamma)))
q = (arw/ad) * 1/gamma
print(f"k: {k} ; q: {q}")
k*np.power(12.5688*np.pi/10000, q)

ldmax: 150.0 ; llwrw: 0.4341683452494336 (avg_llrw: -0.4058316547505664) ; llwd: 1.929419 ; arw: 1.6926673380997734 ; brw: 5.994150439928971 ; ad: 2.2907675999999997 ; bd: 4.2545235869755995


,0,1,2,3,4,5,6,7,8,9
9,0.0866673,0.0911114,0.0938448,0.0946914,0.0936757,0.091006,0.0870408,0.0822034,0.0768893,0.0714159
8,0.100294,0.106736,0.110577,0.111646,0.109984,0.105934,0.100133,0.0933053,0.0860664,0.0788409
7,0.118232,0.127836,0.133319,0.134737,0.131974,0.125581,0.116779,0.106896,0.0968889,0.0872626
6,0.143475,0.156982,0.164993,0.167189,0.162552,0.152009,0.138139,0.1235,0.109555,0.0966836
5,0.179946,0.195886,0.209073,0.213369,0.205616,0.187731,0.165359,0.143423,0.12414,0.107128
4,0.216963,0.2439,0.268543,0.278239,0.265014,0.234443,0.198638,0.166141,0.140354,0.119916
3,0.256476,0.299431,0.347765,0.3702,0.345115,0.290912,0.235189,0.189233,0.156046,0.138432
2,0.287673,0.353452,0.439032,0.48356,0.437219,0.346485,0.266806,0.207635,0.166906,0.145673
1,0.30166,0.380383,0.488206,0.546559,0.486756,0.374262,0.281481,0.215482,0.169884,0.145098
0,0.291707,0.36208,0.452359,0.499609,0.450966,0.356345,0.271729,0.20991,0.162401,0.13094


,0,1,2,3,4,5,6,7,8,9
9,22,23,23,24,23,23,22,20,19,18
8,25,27,28,28,28,27,25,23,21,20
7,30,32,33,34,33,32,29,27,24,22
6,36,40,42,42,41,38,35,31,27,24
5,45,49,53,54,52,47,42,36,31,27
4,55,62,68,70,67,59,50,42,35,30
3,65,76,88,94,88,74,59,48,39,35
2,73,90,111,123,111,88,68,52,42,37
1,76,96,124,139,124,95,71,54,43,37
0,74,92,115,127,114,90,69,53,41,33


k: 0.22701418081032732 ; q: 0.33586743788012635


0.035381536322832294

- **TODO:** Vérifier le ^gamma
- **TODO:** Attention à la somme pour la moyenne

### A Contrast-Based Scalefactor for Luminance Display, G. Ward

#### Version originale 1994

- (L*sf)^1/gamma, ajouté d'après Efficient Radiosity Simulation using Perceptual Metrics and Parallel Processing, S. Gibson

In [18]:
# http://eta-publications.lbl.gov/sites/default/files/lbl-35252.pdf
# A Contrast-based scalefactor for luminance display, G. Ward 1994
lwa = 0.
count = 0
for p in mesh.poly:
    if p.B != 0. and p.rho != -1. :
        lwa += np.log10(p.B)
        count += 1
lwa /= count
lwa += 0.84
lwa = np.power(10, lwa)

print(f"lwa = {lwa}")
ldmax = 150.

gamma = 2.2
sf = (1/ldmax) * np.power((1.219 + np.power(ldmax/2., 0.4)) / (1.219 + np.power(lwa, 0.4)), 2.5)

B_floor_sf = np.zeros([HEIGHT, WIDTH])
for y in range(HEIGHT):
    for x in range(WIDTH):
        B_floor_sf[y, x] = np.power(B_floor[y, x] * sf, 1/gamma)

fdisplay(B_floor_sf, "Floor sf")
data = np.copy(B_floor_sf)
convertRatioToRGB(data)
fakeDisplay("testing...", True)
print(f"lwa: {lwa} ; sf: {sf}")

lwa = 2.7174924442952726


,0,1,2,3,4,5,6,7,8,9
9,0.0797803,0.0853665,0.0888507,0.0899372,0.0886341,0.0852328,0.080246,0.0742703,0.0678478,0.061395
8,0.0972134,0.105759,0.110941,0.112396,0.110137,0.104685,0.0970014,0.0881602,0.0790327,0.0701889
7,0.12146,0.135001,0.142897,0.144958,0.140949,0.131788,0.119445,0.105972,0.0927733,0.0805229
6,0.157824,0.17826,0.190681,0.194122,0.186873,0.17066,0.149933,0.128842,0.109556,0.0925074
5,0.214434,0.240536,0.262708,0.270041,0.256847,0.227085,0.191253,0.157746,0.129746,0.106284
4,0.276214,0.323617,0.368642,0.386768,0.3621,0.306753,0.245121,0.192478,0.153195,0.123807
3,0.346403,0.427164,0.523055,0.569234,0.517668,0.410799,0.308075,0.229548,0.176822,0.150364
2,0.404622,0.534663,0.717002,0.817145,0.712997,0.52045,0.365418,0.260265,0.193679,0.161104
1,0.431474,0.590529,0.827788,0.964447,0.824461,0.577704,0.39288,0.273666,0.198369,0.160245
0,0.41232,0.552403,0.746614,0.854061,0.743505,0.540596,0.374572,0.264132,0.186638,0.139456


,0,1,2,3,4,5,6,7,8,9
9,20,21,22,22,22,21,20,18,17,15
8,24,26,28,28,28,26,24,22,20,17
7,30,34,36,36,35,33,30,27,23,20
6,40,45,48,49,47,43,38,32,27,23
5,54,61,66,68,65,57,48,40,33,27
4,70,82,94,98,92,78,62,49,39,31
3,88,108,133,145,132,104,78,58,45,38
2,103,136,182,208,181,132,93,66,49,41
1,110,150,211,245,210,147,100,69,50,40
0,105,140,190,217,189,137,95,67,47,35


lwa: 2.7174924442952726 ; sf: 0.06749912921610918


## Et maintenant ?
